In [25]:
from dotenv import load_dotenv
load_dotenv()

import pandas as pd

# 파일 저장을 위한 라이브러리
from pathlib import Path

# video statistic(좋조회수/좋아요/쇼츠,롱폼 여부) 함수
from videos import (
    fetch_video_statistics_batch,
    chunked,
)

In [17]:
BASE_DIR = Path("..")        # src 기준
DATA_DIR = BASE_DIR / "data"
INPUT_PATH = DATA_DIR / "channel_all_videos.csv"

df_videos = pd.read_csv(INPUT_PATH)

df_videos.head(), df_videos.shape


(          channel_label   channel_handle     video_id  \
 0  ISEGYE_IDOL_OFFICIAL  isegyeidol_ofcl  YkNTjfc45-o   
 1  ISEGYE_IDOL_OFFICIAL  isegyeidol_ofcl  _KpApEqZp9c   
 2  ISEGYE_IDOL_OFFICIAL  isegyeidol_ofcl  bZHiZn5ea90   
 3  ISEGYE_IDOL_OFFICIAL  isegyeidol_ofcl  im_YVykT91E   
 4  ISEGYE_IDOL_OFFICIAL  isegyeidol_ofcl  -xKhVTH9z3s   
 
                                          video_title          published_at  \
 0  (SUB) 이게 나온다고요?! 😂🎁 랜덤 선물 뽑기 - 화이트 (White) 발매 ...  2026-01-09T09:00:56Z   
 1          이심전심 이세계아이돌 💞 #이세계아이돌 #ISEGYEIDOL #shorts  2026-01-06T09:00:29Z   
 2  저음불가 르르땅 🦊 과 과몰입 징부기 🍔 #이세계아이돌 #ISEGYEIDOL #sh...  2026-01-06T09:00:25Z   
 3   나만 아니면 돼애애애애!!!!!! 🤭 #이세계아이돌 #ISEGYEIDOL #shorts  2026-01-06T09:00:19Z   
 4  (SUB) 게임에 진심인데요...!🔥 운명의 트리 꾸미기 🎄 - 화이트 (White...  2026-01-06T09:00:01Z   
 
    group_label member_label  
 0  ISEGYE_IDOL          NaN  
 1  ISEGYE_IDOL          NaN  
 2  ISEGYE_IDOL          NaN  
 3  ISEGYE_IDOL          NaN  
 4  ISEGYE_IDOL    

In [18]:
video_ids = df_videos["video_id"].dropna().unique().tolist()

len(video_ids)

10371

In [19]:
video_stats_map = {}

for chunk in chunked(video_ids, size=50):
    batch_stats = fetch_video_statistics_batch(chunk)
    video_stats_map.update(batch_stats)

len(video_stats_map)

10371

In [20]:
stats_df = (
    pd.DataFrame
    .from_dict(video_stats_map, orient="index")
    .reset_index()
    .rename(columns={"index": "video_id"})
)

stats_df.head()

,video_id,view_count,like_count,comment_count,duration_sec,video_type
0,YkNTjfc45-o,27263,2913,155,641.0,LONG
1,_KpApEqZp9c,128540,7014,142,48.0,SHORT
2,bZHiZn5ea90,57618,4515,110,57.0,SHORT
3,im_YVykT91E,60558,4609,127,59.0,SHORT
4,-xKhVTH9z3s,32760,3396,213,732.0,LONG


In [21]:
df_videos_full = df_videos.merge(
    stats_df,
    on="video_id",
    how="left"
)

df_videos_full.head(), df_videos_full.shape

(          channel_label   channel_handle     video_id  \
 0  ISEGYE_IDOL_OFFICIAL  isegyeidol_ofcl  YkNTjfc45-o   
 1  ISEGYE_IDOL_OFFICIAL  isegyeidol_ofcl  _KpApEqZp9c   
 2  ISEGYE_IDOL_OFFICIAL  isegyeidol_ofcl  bZHiZn5ea90   
 3  ISEGYE_IDOL_OFFICIAL  isegyeidol_ofcl  im_YVykT91E   
 4  ISEGYE_IDOL_OFFICIAL  isegyeidol_ofcl  -xKhVTH9z3s   
 
                                          video_title          published_at  \
 0  (SUB) 이게 나온다고요?! 😂🎁 랜덤 선물 뽑기 - 화이트 (White) 발매 ...  2026-01-09T09:00:56Z   
 1          이심전심 이세계아이돌 💞 #이세계아이돌 #ISEGYEIDOL #shorts  2026-01-06T09:00:29Z   
 2  저음불가 르르땅 🦊 과 과몰입 징부기 🍔 #이세계아이돌 #ISEGYEIDOL #sh...  2026-01-06T09:00:25Z   
 3   나만 아니면 돼애애애애!!!!!! 🤭 #이세계아이돌 #ISEGYEIDOL #shorts  2026-01-06T09:00:19Z   
 4  (SUB) 게임에 진심인데요...!🔥 운명의 트리 꾸미기 🎄 - 화이트 (White...  2026-01-06T09:00:01Z   
 
    group_label member_label  view_count  like_count  comment_count  \
 0  ISEGYE_IDOL          NaN       27263        2913            155   
 1  ISEGYE_IDOL          NaN    

In [22]:
# Unknown 1개 실시간 스트리밍 영상
df_videos_full["video_type"].value_counts()

video_type
LONG       5289
SHORT      5077
UNKNOWN       5
Name: count, dtype: int64

In [24]:
BASE_DIR = Path("..")        # src 기준
DATA_DIR = BASE_DIR / "data"
OUTPUT_PATH = DATA_DIR / "channel_videos_with_stats.csv"

df_videos_full.to_csv(OUTPUT_PATH, index=False)

print(f"Saved to {OUTPUT_PATH}")

Saved to ..\data\channel_videos_with_stats.csv


In [10]:
df_videos_full[df_videos_full["video_type"]=="UNKNOWN"]

,channel_label,channel_handle,video_id,video_title,published_at,group_label,member_label,view_count,like_count,comment_count,duration_sec,video_type
2125,LILPA,lilpa,9Zwrr5J3XaA,릴판다 홈쇼핑! 지금 바로 시작합니다!,2023-09-28T13:29:40Z,ISEGYE_IDOL,LILPA,0,10,0,NaN,UNKNOWN
3787,GOSEGU,gosegu,9t3NG2Ivzgw,고튜브 50만 기념! 팬분과 영상통화하기,2024-03-22T13:04:48Z,ISEGYE_IDOL,GOSEGU,0,12,0,NaN,UNKNOWN
7394,PLAVE,plave_official,Xm5QIEoV8TU,260213 PLAVE(플레이브) Live Broadcast 📡,2026-02-08T06:04:14Z,PLAVE,NaN,0,234,0,NaN,UNKNOWN
7395,PLAVE,plave_official,OyzcBQqoixg,260210 PLAVE(플레이브) NOAH Birthday Live Broadcast 📡,2026-02-08T06:02:28Z,PLAVE,NaN,0,265,0,NaN,UNKNOWN
8697,PLAVE,plave_official,VUNJR6D7naE,플레이브 확신의 개그캐(?) 연습생 노아의 첫 번째 라이브 풀버전💜｜#플레이브 PL...,2023-04-27T11:05:21Z,PLAVE,NaN,0,1,0,NaN,UNKNOWN
